<a href="https://colab.research.google.com/github/Saleha321/General-Health-Query-Chatbot/blob/main/Health_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate bitsandbytes

In [2]:
from huggingface_hub import login

# This will prompt you to enter your Hugging Face token securely.
login()

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Define the model name
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

print(f"Loading tokenizer for {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Tokenizer loaded.")

print(f"Loading model for {model_name}...")
# Load the model in 4-bit to save memory
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True, # This is crucial for running on Colab's free GPU
    device_map="auto",
    torch_dtype=torch.float16 # Use float16 for better performance and lower memory usage
)
print("Model loaded.")

# Set the model to evaluation mode
model.eval()

Loading tokenizer for mistralai/Mistral-7B-Instruct-v0.2...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Tokenizer loaded.
Loading model for mistralai/Mistral-7B-Instruct-v0.2...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded.


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

In [4]:
def get_health_response(query):
    # --- Prompt Engineering ---
    # System message to set the persona and guidelines
    system_message = (
        "You are a helpful and friendly medical assistant. "
        "You provide general health information and explanations, "
        "but **NEVER give specific medical advice, diagnoses, or prescribe treatments.** "
        "Always advise users to consult a qualified healthcare professional for personalized medical advice. "
        "Keep your answers clear, concise, and easy to understand. "
        "If a question asks for specific medical advice or a diagnosis, politely state that you cannot provide that and recommend seeing a doctor."
    )

    # User's query
    user_query = query

    # Combine system message and user query for the model
    # Mistral uses a specific format for its instructions, known as "instruction tuning"
    # The prompt template is important for getting good responses from Mistral
    prompt = f"<s>[INST] {system_message}\n\n{user_query} [/INST]"

    # --- Safety Filters (Rule-based for now) ---
    # These are basic, rule-based filters. For robust safety, more advanced techniques
    # like content moderation APIs or fine-tuning models on safety data are used.
    lower_query = query.lower()
    if "diagnose" in lower_query or \
       "prescribe" in lower_query or \
       "treatment for" in lower_query or \
       "should i take" in lower_query or \
       "what drug" in lower_query or \
       "am i having" in lower_query or \
       "is it cancer" in lower_query:
        return "I am a medical assistant and cannot provide medical advice, diagnoses, or prescribe treatments. Please consult a qualified healthcare professional for personalized medical guidance."

    # --- Generate Response from LLM ---
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    # Generate the response
    with torch.no_grad(): # Disable gradient calculation for inference, saves memory and speeds up
        outputs = model.generate(
            **inputs,
            max_new_tokens=500, # Maximum number of tokens to generate in the response
            temperature=0.7,    # Controls randomness: lower for more deterministic, higher for more creative
            do_sample=True,     # Enable sampling for more varied responses
            top_k=50,           # Consider only the top-k most likely tokens
            top_p=0.95,         # Nucleus sampling: consider tokens until cumulative probability reaches top_p
            pad_token_id=tokenizer.eos_token_id # Important for handling padding
        )

    # Decode the generated tokens back to text
    # We need to slice the output to remove the input prompt
    response_text = tokenizer.decode(outputs[0][len(inputs["input_ids"][0]):], skip_special_tokens=True)

    # --- Post-processing for safety and clarity ---
    # Add a general disclaimer at the end
    if "Please consult a qualified healthcare professional" not in response_text:
        response_text += "\n\n**Disclaimer:** This information is for general knowledge and informational purposes only, and does not constitute medical advice. Please consult a qualified healthcare professional for any health concerns or before making any decisions related to your health or treatment."

    return response_text.strip()

In [ ]:
print("Welcome to the General Health Chatbot! (Type 'quit' to exit)")

while True:
    user_input = input("\nYou: ")
    if user_input.lower() == 'quit':
        print("Chatbot: Goodbye!")
        break

    print("Chatbot (thinking...): ", end="")
    response = get_health_response(user_input)
    print(response)

Welcome to the General Health Chatbot! (Type 'quit' to exit)

You: What causes a sore throat?
Chatbot (thinking...): 

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:463: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


A sore throat is often caused by infections such as the common cold or flu, strep throat, or other viruses. Irritants like acid reflux, smoking, or screaming or shouting can also cause a sore throat. Other potential causes include bacterial infections, allergies, or dry environments. It's important to note that a sore throat can also be a symptom of more serious conditions, so if you have persistent symptoms, please consult a healthcare professional for proper evaluation and treatment.

**Disclaimer:** This information is for general knowledge and informational purposes only, and does not constitute medical advice. Please consult a qualified healthcare professional for any health concerns or before making any decisions related to your health or treatment.
